In [4]:


from imutils import paths
from sklearn.datasets import load_sample_image
import matplotlib.pyplot as plt

from skimage.feature import greycomatrix, greycoprops
from skimage import data
import numpy as np

from skimage import io, color, img_as_ubyte
import json
import csv

from PIL import Image, ImageDraw

ORIG_INPUT_DATASET = "datasets/orig"
image_paths = list(paths.list_images(ORIG_INPUT_DATASET))
PATCH_SIZE = 10


def do_stuff(image, class_number):
    # select some patches from dot1y areas of the image
    x1, y1 = (198, 294)
    x2, y2 = (200, 390)
    offset = 10

    dot1_locations = [(x1 - offset, y1 - offset), (x1 - offset, y1), (x1 - offset, y1 + offset),
                      (x1, y1 - offset), (x1, y1), (x1, y1 + offset),
                      (x1 + offset, y1 - offset), (x1 + offset, y1), (x1 + offset, y1 + offset)]
    dot1_patches = []
    for loc in dot1_locations:
        dot1_patches.append(image[loc[0]:loc[0] + PATCH_SIZE,
                                  loc[1]:loc[1] + PATCH_SIZE])

    # select some patches from dot2 areas of the image
    dot2_locations = [(x2 - offset, y2 - offset), (x2 - offset, y2), (x2 - offset, y2 + offset),
                      (x2, y2 - offset), (x2, y2), (x2, y2 + offset),
                      (x2 + offset, y2 - offset), (x2 + offset, y2), (x2 + offset, y2 + offset)]
    dot2_patches = []
    for loc in dot2_locations:
        dot2_patches.append(image[loc[0]:loc[0] + PATCH_SIZE,
                                  loc[1]:loc[1] + PATCH_SIZE])

    # compute some GLCM properties each patch

    xs = []
    ys = []
    
    zs = []
    #ls = []
    #ms = []
    #ns = []

    for patch in (dot1_patches + dot2_patches):
        glcm = greycomatrix(patch, distances=[1,2,3], angles=[
            0, np.pi/4, 3*np.pi/4], levels=256,
            symmetric=True, normed=True)
        xs.append(greycoprops(glcm, 'dissimilarity')[0, 0])
        ys.append(greycoprops(glcm, 'correlation')[0, 0])

        zs.append(greycoprops(glcm, 'contrast')[0, 0])
        #ls.append(greycoprops(glcm, 'homogeneity')[0, 0])
        #ms.append(greycoprops(glcm, 'ASM')[0, 0])
        #ns.append(greycoprops(glcm, 'energy')[0, 0])

    aux_list = list(zip(
        xs[:len(dot1_patches)], ys[:len(dot1_patches)], zs[:len(dot1_patches)], 
        #ls[:len(dot1_patches)], ms[:len(dot1_patches)], ns[:len(dot1_patches)],
        [1] * len(dot1_patches), 
        [class_number] * len(dot1_patches)
    ))

    aux_list += list(zip(
        xs[len(dot2_patches):], ys[len(dot2_patches):], zs[:len(dot1_patches)],
        #ls[:len(dot1_patches)], ms[:len(dot1_patches)], ns[:len(dot1_patches)],
        [2] * len(dot2_patches), 
        [class_number] * len(dot2_patches)
    ))

    for x in aux_list:
        data.append(x)



data = []

for image_path in image_paths:
    image = io.imread(image_path, as_gray=True)
    do_stuff(image, int(image_path[-5:-4]))

with open("data.txt", "w") as f:
    json.dump(data, f)


fields = ['dissimilarity', 'correlation', 'contrast', 'dot', 'class']


with open('data.csv', 'w') as f:

    # using csv.writer method from CSV package
    write = csv.writer(f)

    write.writerow(fields)
    write.writerows(data)
